In [3]:
#연습문제 : www.yelp.com 이용 후기 정보 수집하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib
import time
import sys
import re
import math
import time
import os
import random
import pandas as pd

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.

print("=" *80)
print(" 이 크롤러는 www.yelp.com의 레스트랑 이용 후기 정보를 수집합니다")
print("=" *80)

query_txt = input('1.크롤링할 메뉴는 무엇입니까?(예:sushi): ')
query_area = input('2.크롤링할 지역은 어디 입니까?(예:LA):')

m_cnt = int(input('3.크롤링 할 매장수는 몇개입니까?: '))
#m_page_cnt = math.ceil(cnt / 30)

r_cnt = int(input('4.각 매장별 크롤링할 리뷰 개수는 몇개입니까?: '))
#r_page_cnt = math.ceil(cnt / 20)

f_dir=input('5.파일이 저장될 경로만 쓰세요(예: c:\\temp\\ ) : ')
if f_dir =='' :
        f_dir = "c:\\temp\\"

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)


os.makedirs(f_dir+s+'-'+query_txt+'-'+query_area)

ff_name=f_dir+s+'-'+query_txt+'-'+query_area+'\\'+s+'-'+query_txt+'-'+query_area+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+query_area+'\\'+s+'-'+query_txt+'-'+query_area+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+query_area+'\\'+s+'-'+query_txt+'-'+query_area+'.xls'
        
#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)


driver.get('https://www.yelp.com')
time.sleep(random.randrange(2,5))

driver.find_element_by_link_text("Restaurants").click( )
time.sleep(random.randrange(2,5))

menu = driver.find_element_by_id("search_description")
driver.find_element_by_id("search_description").click()
menu.clear()
time.sleep(1)
menu.send_keys(query_txt)
time.sleep(1)


area = driver.find_element_by_id("search_location")
driver.find_element_by_id("search_location").click()
time.sleep(1)
area.send_keys(query_area)
time.sleep(1)

driver.find_element_by_xpath('''//*[@id="header_find_form"]/div/div[2]/div/div[2]/button''').click( )
time.sleep(10)

# 스크롤 다운 함수를 만듭니다
def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)



scroll_down(driver)

ranking2=[]
title3=[]
shop_info2=[]
menu2=[]
score2=[]
review_cnt2=[]

file_no = 1

# 이미지 저장용 폴더 생성하기
img_dir = f_dir+s+'-'+query_txt+'-'+query_area+'\\'+'images'
os.makedirs(img_dir)
os.chdir(img_dir)


# 이미지 추출 코드 추가
img_src2=[]   # 이미지 URL 저장변수
file_no = 1

click_count = 1   # 페이지 번호 계산용 변수
m_count = 1       # 매장 개수 계산용 변수

if m_cnt < 31 :
    m_page_cnt = 1 
else :
    m_page_cnt = math.ceil(m_cnt/30)
    
m_url_list=[]   # 정보를 수집할 매장의 원본 URL 주소를 저장하는 변수    

    
while ( click_count <= m_page_cnt ) :
    html = driver.page_source
    soup_1 = BeautifulSoup(html, 'html.parser')

    print("\n")

    re_result_1 = soup_1.find_all('li','lemon--li__373c0__1r9wz border-color--default__373c0__2oFDT')
    re_result_2 = re_result_1[29:66]
    
    for li in re_result_2:
            
            # 이미지 저장하기
            try :
                photo = li.find('div','lemon--div__373c0__1mboc on-click-container border-color--default__373c0__2oFDT').find('img')['src']
            except  :
                #print("사진이 없습니다")
                continue
            else :
                urllib.request.urlretrieve(photo,str(file_no)+'.jpg')               
                print("%s 번째 그림 다운로드 했습니다" %file_no)
                
                time.sleep(random.randrange(3,10))

                if r_cnt < file_no :
                    break
                file_no += 1
            
            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")
            
            print("%s 번째 게시글의 정보를 수집합니다" %m_count)
            # 판매순위
            print("-" *70)
            try :
                rank_1 = li.find('h3','lemon--h3__373c0__sQmiG heading--h3__373c0__1n4Of alternate__373c0__1uacp').get_text()
                rank_2 = rank_1.split('.')
                ranking = rank_2[0]
            except  :
                ranking = '순위없음'
                print(ranking)
            else :
                print("1.판매순위:",ranking)
                ranking2.append(ranking)

                f.write('1.판매순위:'+ ranking + "\n")
            

            #매장명 
            try :
                rank_3 = li.find('h3','lemon--h3__373c0__sQmiG heading--h3__373c0__1n4Of alternate__373c0__1uacp').get_text()
                rank_4 = rank_3.split('.')
                title = rank_4[1]
            except  :
                title1 = '매장명없음'
                print(title1.strip( ))
                f.write('2.매장명:'+ title1 + "\n")
            else :
                #title2=title1.translate(bmp_map).replace("\n","") 
                print("2.매장명:", title.strip())
                title3.append(title)                             
                f.write('2.매장명:'+ title + "\n")
                
           # 매장 전화번호와 주소 

            try :
                tel_addr = li.find('div','lemon--div__373c0__1mboc container__373c0__19wDx u-padding-l2 border-color--default__373c0__2oFDT text-align--right__373c0__3fmmn') 
                shop_info = tel_addr.get_text()
            except :
                shop_info = '전화번호가 없습니다'
            else :
                print('3.전화번호와 주소:' , shop_info)
                shop_info2.append(shop_info)
                f.write('3.전화번호와 주소:'+ shop_info + "\n")
                        
            # 판매 메뉴

            menu_1 = li.find('div','lemon--div__373c0__1mboc priceCategory__373c0__3zW0R border-color--default__373c0__2oFDT').get_text()
            #print(menu_1.get_text())
            print('4.주메뉴:' , menu_1)
            menu2.append(menu_1)
            f.write('4.주메뉴:'+ menu_1 + "\n")
    
            
             #상품 별점 구하기
                   
            score_1 = li.find('div','lemon--div__373c0__1mboc attribute__373c0__1hPI_ display--inline-block__373c0__2de_K u-space-r1 border-color--default__373c0__2oFDT').find('div')
            #score_1 = li.find('div','lemon--div__373c0__1mboc i-stars__373c0__30xVZ i-stars--regular-4-half__373c0__QMLlH border-color--default__373c0__2oFDT overflow--hidden__373c0__8Jq2I')
            #print(score_1)
            
            score = score_1['aria-label']

            print('5.평점:',score)
            score2.append(score)
            f.write('5.평점:'+ score + "\n")           

            # 상품평 수
    
            try :
                sat_count = li.find('span','lemon--span__373c0__3997G text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_').get_text()
            except (IndexError , AttributeError) :
                sat_count='0'
                print('6.상품평 수: ',sat_count)
                f.write('6.상품평 수:'+ sat_count + "\n")
            else :
                sat_count_2 = sat_count.replace(",","")
                sat_count_3 = re.search("\d+",sat_count_2)
                review_cnt = int(sat_count_3.group())
                print('6.상품평 수:',review_cnt)
                review_cnt2.append(review_cnt)
                f.write('6.상품평 수:'+ str(review_cnt) + "\n")

            
            #매장 URL 주소 추출
            
            try:
                url_2 = li.find('a')['href']
            except :
                continue
            else :  
                url_1 = 'https://www.yelp.com'
                full_url = url_1 + url_2
                time.sleep(0.5)
                m_url_list.append(full_url)
                print('7.%s번째 매장 URL:' %m_count , full_url) 
                f.write('7.URL주소:'+ full_url + "\n")
            
            print("=" *80)
            print("\n")
            
            time.sleep(random.randrange(3,10))
            
            if m_count == m_cnt :
                break
                
            m_count += 1
                       
    print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(click_count,m_count))
                    
    if click_count > m_page_cnt :
            break
    else :
        click_count += 1
        driver.find_element_by_link_text("%s" %click_count).click()
                
    time.sleep(random.randrange(10,20))         

 #Step 5. 검색 결과를 다양한 형태로 저장하기
              
yelf = pd.DataFrame()
yelf['판매순위']=ranking2
yelf['매장명']=pd.Series(title3)
yelf['전화번호와 주소']=pd.Series(shop_info2)
yelf['주메뉴']=pd.Series(menu2)
yelf['상품평점']=pd.Series(score2)
yelf['상품평수']=pd.Series(review_cnt2)
yelf['URL주소']=pd.Series(m_url_list)


# csv 형태로 저장하기
yelf.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
yelf.to_excel(fx_name ,index=True) 

#################################################################################
#  여기서부터는 각 매장별 리뷰를 수집하는 구간입니다
# 각 매장별 리뷰를 수집할 용도로 검색할 매장의 목록을 만듭니다
#################################################################################
print("\n")
print("=" *80)
print("지금부터 각 매장별 리뷰 수집을 시작합니다")
print("리뷰의 내용이 많아서 다소 시간이 소요될 수 있으니 잠시만 기다려 주세요~~~^^")
print("리뷰 수집을 요청하신 매장수는 총 %s 개 이고 각 매장별로 %s 건의 리뷰 수집을 요청하셨습니다~^^" %(m_cnt,r_cnt))
print("=" *80)
print("\n")

r_page_cnt = math.ceil(review_cnt/20)



no = 1
#for r in range(0,len(review_cnt2)) :
for r in range(0,len(m_url_list)) :
    if r_cnt > review_cnt2[r] :
        r_cnt = review_cnt2[r]
        print("\n")
        print("수집을 요청하신 리뷰 건수는 %s 건이었으나 실제 리뷰 건수가 %s건이라서 %s건으로 수집하겠습니다" %(r_cnt,review_cnt2[r]))
    else :
        print("\n")
        print("수집을 요청하신 리뷰 %s 건의 수집을 시작합니다~^^ " %r_cnt)
        
        # 해당 매장의 상세 페이지를 엽니다
        #for f in range(0,len(m_url_list)) :
        print("현재 총 %s개의 매장 중 %s번째 매장인 %s 매장의 리뷰를 수집중입니다~~ " %(m_cnt,no,title3[r]))
        print("참고로 이 매장의 총 리뷰 갯수는 현재 %s 개 입니다~" %str(review_cnt2[r]))
                
        os.makedirs(f_dir+s+'-'+query_txt+'-'+query_area+'\\'+title3[r])
        os.chdir(f_dir+s+'-'+query_txt+'-'+query_area+'\\'+title3[r])
        
        r_ff_name=f_dir+s+'-'+query_txt+'-'+query_area+'\\'+title3[r]+'\\'+s+'-'+query_txt+'-'+query_area+'-'+title3[r]+'.txt'
        r_fc_name=f_dir+s+'-'+query_txt+'-'+query_area+'\\'+title3[r]+'\\'+s+'-'+query_txt+'-'+query_area+'-'+title3[r]+'.csv'
        r_fx_name=f_dir+s+'-'+query_txt+'-'+query_area+'\\'+title3[r]+'\\'+s+'-'+query_txt+'-'+query_area+'-'+title3[r]+'.xls'
        
        user_name2 =[]    #리뷰 작성자 이름
        loc2 = []         #리뷰 작성자 위치
        point2 =[]        #리뷰 작성자 점수
        r_date2=[]        #리뷰작성일자
        r_content2=[]    #리뷰 내용

        driver.get(m_url_list[r])
        time.sleep(random.randrange(10,15))
        no_cnt = 1           # 게시물 건수 체크 변수
                   
        # 해당 식당의 리뷰 내용 수집                
        for e in range(2,r_page_cnt + 1) :
                
                    # 현재 페이지의 리뷰 정보 수집하기
                    html = driver.page_source
                    soup2 = BeautifulSoup(html, 'html.parser')
                        
                    # 1. 이름 / 2. 지역 / 3. 평점 / 4.리뷰작성일자/ 5.리뷰내용 / 6. 사진
                                
                    re_1 = soup2.find('ul','ylist ylist-bordered reviews').find_all('li')
                    re_2 = re_1[6:]
               
                    #print(re_2[0].get_text())
                    
                    for x in range(0,len(re_2)) :
                        # 1. 이름
                        try :
                            user_name = re_2[x].find('li','user-name').select('#dropdown_user-name')[0].get_text()
                        except :
                            continue
                        else :
                            print("\n")
                            print( '%s번째 매장의 %s번째 리뷰 정보를 수집합니다---------------------------' %(no,no_cnt))
                            rf = open(r_ff_name, 'a',encoding='UTF-8')
                            rf.write("-----------------------------------------------------"+"\n")
                            rf.write('%s번째 리뷰 정보를 수집합니다------------------------' %no + "\n")
                            
                            print('1.리뷰작성자명:',user_name)
                            user_name2.append(user_name)
                            rf.write('1.리뷰작성자명:'+ user_name + "\n")
                        
                        #2. 리뷰 작성자 위치
                        try :
                            loc = re_2[x].find('li','user-location responsive-hidden-small').get_text().replace("\n","")
                        except :
                            continue
                        else :
                            print('2.리뷰작성지역:',loc)
                            loc2.append(loc)
                            rf.write('2.리뷰작성지역:'+ loc + "\n")                    

                        # 3.리뷰 평점
                        point_1 = re_2[x].select('div.biz-rating__stars > div')
                        point_2 = point_1[0]['title']

                        print('3.리뷰평점:',point_2)
                        point2.append(point_2)
                        rf.write('3.리뷰평점:'+ point_2 + "\n")

                    
                        # 4.리뷰 작성 일자
                        r_date1 = re_2[x].find('div','biz-rating').get_text().replace("\n","").strip()
                        print('4.리뷰작성일자:',r_date1)
                        r_date2.append(r_date1)
                        rf.write('4.리뷰작성일자:'+r_date1 + "\n") 
                    
                        #5. 리뷰 내용
                        r_content1 = re_2[x].find('p').get_text().replace("\n","").strip()
                        print('5.리뷰내용:',r_content1)
                        r_content2.append(r_content1)
                        rf.write('5.리뷰내용:'+r_content1 + "\n")
                    
                                                
                        rf.close()
                        
                        if no_cnt == r_cnt :
                            break
                            
                        no_cnt += 1
                    
                        time.sleep(random.randrange(1,3))        # 페이지 변경 전 2초 대기 

    
                    if e > r_page_cnt :
                        break              
                    elif no_cnt == r_cnt :
                        break
                    elif no_cnt > review_cnt :
                        break
                    else :
                        driver.find_element_by_link_text("""%s""" %e).click() # 다음 페이지번호 클릭
                        time.sleep(5)
        
                    yelf_review = pd.DataFrame()
                    yelf_review['리뷰작성자명']=pd.Series(user_name2)
                    yelf_review['리뷰작성지역']=pd.Series(loc2)
                    yelf_review['리뷰평점']=pd.Series(point2)
                    yelf_review['리뷰작성일자']=pd.Series(r_date2)
                    yelf_review['리뷰내용']=pd.Series(r_content2)


                    # csv 형태로 저장하기
                    yelf_review.to_csv(r_fc_name,encoding="utf-8-sig",index=True)

                    # 엑셀 형태로 저장하기
                    yelf_review.to_excel(r_fx_name ,index=True)
                        
        print("이 페이지의 리뷰 수집이 끝났습니다")
        print("\n")
    no += 1

 이 크롤러는 www.yelp.com의 레스트랑 이용 후기 정보를 수집합니다
1.크롤링할 메뉴는 무엇입니까?(예:sushi): burgers
2.크롤링할 지역은 어디 입니까?(예:LA):LA
3.크롤링 할 매장수는 몇개입니까?: 2
4.각 매장별 크롤링할 리뷰 개수는 몇개입니까?: 25
5.파일이 저장될 경로만 쓰세요(예: c:\temp\ ) : c:\data\


1 번째 그림 다운로드 했습니다
1 번째 게시글의 정보를 수집합니다
----------------------------------------------------------------------
1.판매순위: 1
2.매장명: Cassell’s Hamburgers
3.전화번호와 주소: (213) 387-55023600 W 6th StWilshire Center
4.주메뉴: $$American (Traditional), Burgers, Breakfast & Brunch
5.평점: 4 star rating
6.상품평 수: 1443
7.1번째 매장 URL: https://www.yelp.com/biz/cassells-hamburgers-los-angeles-2?osq=burgers


2 번째 그림 다운로드 했습니다
2 번째 게시글의 정보를 수집합니다
----------------------------------------------------------------------
1.판매순위: 2
2.매장명: Golden State
3.전화번호와 주소: (323) 782-8331426 N Fairfax AveFairfax
4.주메뉴: $$Burgers, American (Traditional), Beer Bar
5.평점: 4 star rating
6.상품평 수: 1498
7.2번째 매장 URL: https://www.yelp.com/biz/golden-state-los-angeles-3?osq=burgers


1 번째 페이지까지 2건 정보 수집 완료 ====================


지금부터 각 매장



1번째 매장의 9번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Chase P.
2.리뷰작성지역: Beverly Hills, CA
3.리뷰평점: 5.0 star rating
4.리뷰작성일자: 6/2/2019            Updated review
5.리뷰내용: Came back for another BOMB meal! I got the b'fast burger and it was heaven. The hash brown was cooked perfectly, not too crisp but had a little crunch to it. I didn't think the texture of the hash brown would be this good so it definitely exceeded expectations. The burger was cooked well done and the seasonings made it so delicious. They add plenty of avocado on the burger and it was fresh! Everything about this dish was amazing and I would get it again, for sure! I also tried their pie this time. I got the dulce de leche banana (banana and caramel) and OH MY DAMN you have to get this! Best pie ever, no lie. I'm not really a pie person but yelp hyped it up and it definitely lived up to the hype.Always a good experience at Cassell's! Shannon was our waiter this time and she was super nice! Very attentive and qui



1번째 매장의 19번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Marc B.
2.리뷰작성지역: Poway, CA
3.리뷰평점: 5.0 star rating
4.리뷰작성일자: 4/30/2019
5.리뷰내용: Enjoyed my cheeseburger! Highlights: thickness of the meat, bun, and housemade sauces (Thousand Island and Spicy Mayo in particular). Everything but the burger and cheese are served on the side, so you can stack it as you please. It came out medium-well even though I ordered it medium, which meant that it wasn't as juicy.  I also had a strawberry shake was with malt. The shake was huge, which I super appreciated, and the malt gave it some needed thickness. Great interior and lots of stuff on the walls.


1번째 매장의 20번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Jess S.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 4.0 star rating
4.리뷰작성일자: 4/29/2019
5.리뷰내용: Great place to come for your standard burger. The cheeseburger here isn't super greasy and I love the buns they use. You can customize your burger with toppings for additional cost and they also h



2번째 매장의 2번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Marcia T.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 4.0 star rating
4.리뷰작성일자: 7/7/2019
5.리뷰내용: Yummy yummy burgers.  Burgers are seasoned well and juicy.  The atmosphere here is great, makes me feel like I'm in a New York cool, hip restaurant, it's decorated great and simple.   I like the guys who work there also. The cashier, the cooks, etc.  Oh yeah, I'm from Wisconsin and I think we have the best cheese and bread of all time, but this place... the bread on the buns are delicious and soft, just the way I like them.


2번째 매장의 3번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Shannon N.
2.리뷰작성지역: West Hollywood, CA
3.리뷰평점: 3.0 star rating
4.리뷰작성일자: 6/3/2019
5.리뷰내용: Mixed thoughts about Golden State. I'm trying to limit my intake of burgers and decided to try this place on a whim. Located across from Canter's, you'll understand this area has crap parking (metered street but it's always taken) or if you're lucky, some neighborho



2번째 매장의 14번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Stefanny B.
2.리뷰작성지역: 
3.리뷰평점: 4.0 star rating
4.리뷰작성일자: 11/10/2018
5.리뷰내용: The burger was reallyGood probably one of the best burgers I've had. But very pricey. I didn't like their lemonade it was too minty for me.   Next time I'll have a beer it looks like they have a great selection of beers. Very friendly staff.


2번째 매장의 15번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Jeanne N.
2.리뷰작성지역: Malibu, CA
3.리뷰평점: 5.0 star rating
4.리뷰작성일자: 7/24/2019
5.리뷰내용: Just ate here today. I treated myself to their Hamburger and it was amazing.  The soft brioche bread melted with the cheese and juices.  I'm dreaming about it now. I don't eat a lot of beef so this treat was so worth it!  My daughter had the veggie burger and sweet potato fries.  She said the flavors were amazing We both had their ice tea -  perfect brew and needed nothing.Two thumbs way up.


2번째 매장의 16번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: Nicole G.



2번째 매장의 25번째 리뷰 정보를 수집합니다---------------------------
1.리뷰작성자명: David R.
2.리뷰작성지역: Los Angeles, CA
3.리뷰평점: 5.0 star rating
4.리뷰작성일자: 6/17/2019
5.리뷰내용: I got a burger craving at like 9pm a couple weeks back and decided to take a chance on Golden State.  I ordered "The burger" delivery.  What I got was an unexpected marriage of all the right decisions.  The meat was cooked and seasoned a perfect med rare, the cheese and bacon were of impeccable quality, and the aioli was a perfect foil to the micro greens and softly toasted brioche.  I ordered another one delivery less than a week later, and it was just as good.  If a burger is this stupefyingly amazing after being driven across town, I can't wait to sit down and have one fresh in front of me.  The fries were legit as hell too fam.
이 페이지의 리뷰 수집이 끝났습니다




In [4]:
#Step 5. 검색 결과를 다양한 형태로 저장하기
              
#yelf_review = pd.DataFrame()
#yelf_review['리뷰작성자명']=pd.Series(user_name2)
#yelf_review['리뷰작성지역']=pd.Series(loc2)
#yelf_review['리뷰평점']=pd.Series(point2)
#yelf_review['리뷰작성일자']=pd.Series(r_date2)
#yelf_review['리뷰내용']=pd.Series(r_content2)


# csv 형태로 저장하기
#yelf_review.to_csv(r_fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
#yelf_review.to_excel(r_fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time


#이미지 삽입하기 - 전체 매장 목록 엑셀파일임 / 리뷰 파일 아님

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = m_cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,m_cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)

for b in range(1,m_cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,m_cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()

# Step 6. 요약 정보를 출력하기


print("\n") 
print("=" *80)
#print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("3.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("4.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("5.리뷰파일 저장 완료: txt 파일명 : %s " %r_ff_name)
print("6.리뷰파일 저장 완료: csv 파일명 : %s " %r_fc_name)
print("7.리뷰파일 저장 완료: xls 파일명 : %s " %r_fx_name)
print("=" *80)    

driver.close()



1.총 소요시간은 249.5 초 입니다 
2.파일 저장 완료: txt 파일명 : c:\data\2019-08-01-14-57-16-burgers-LA\2019-08-01-14-57-16-burgers-LA.txt 
3.파일 저장 완료: csv 파일명 : c:\data\2019-08-01-14-57-16-burgers-LA\2019-08-01-14-57-16-burgers-LA.csv 
4.파일 저장 완료: xls 파일명 : c:\data\2019-08-01-14-57-16-burgers-LA\2019-08-01-14-57-16-burgers-LA.xls 
5.리뷰파일 저장 완료: txt 파일명 : c:\data\2019-08-01-14-57-16-burgers-LA\ Golden State\2019-08-01-14-57-16-burgers-LA- Golden State.txt 
6.리뷰파일 저장 완료: csv 파일명 : c:\data\2019-08-01-14-57-16-burgers-LA\ Golden State\2019-08-01-14-57-16-burgers-LA- Golden State.csv 
7.리뷰파일 저장 완료: xls 파일명 : c:\data\2019-08-01-14-57-16-burgers-LA\ Golden State\2019-08-01-14-57-16-burgers-LA- Golden State.xls 
